# Dependencies

In [1]:
! pip install starknet-py

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


# Converter

In [16]:
import numpy as np
from starknet_py.hash.utils import pedersen_hash

def float_to_fixed_point(value, integer_bits, fractional_bits):
    scale_factor = 2**fractional_bits
    return f"FP{integer_bits}x{fractional_bits} {{ mag: {abs(int(value * scale_factor))}, sign: {str(value < 0).lower()} }}"

def cairo_array(arr, type_name="usize", fixed_point_type="FP16x16"):
    if type_name.startswith("FP"):
        integer_bits, fractional_bits = map(int, fixed_point_type[2:].split('x'))
        return ", ".join([float_to_fixed_point(x, integer_bits, fractional_bits) for x in arr])
    return ", ".join(map(str, arr))

class TreeEnsembleAttributes:
    def __init__(self):
        self._names = []

    def add(self, name, value):
        if not name.endswith("_as_tensor"):
            self._names.append(name)
        if isinstance(value, list):
            dtype = np.float32 if name in {"base_values", "class_weights", "nodes_values", "nodes_hitrates"} else None
            value = np.array(value, dtype=dtype)
        setattr(self, name, value)

class TreeEnsemble:
    def __init__(self, fixed_point_type="FP16x16", **kwargs):
        self.atts = TreeEnsembleAttributes()
        self.fixed_point_type = fixed_point_type
        for name, value in kwargs.items():
            self.atts.add(name, value)

        self.tree_ids = sorted(set(self.atts.nodes_treeids))
        self._initialize_indices()

    def _initialize_indices(self):
        self.root_index = {tid: len(self.atts.nodes_treeids) for tid in self.tree_ids}
        for index, tree_id in enumerate(self.atts.nodes_treeids):
            self.root_index[tree_id] = min(self.root_index[tree_id], index)
        self.node_index = {(tid, nid): i for i, (tid, nid) in enumerate(zip(self.atts.nodes_treeids, self.atts.nodes_nodeids))}

    def generate_cairo_code(self):
        tree_ids_cairo = f"let tree_ids: Span<usize> = array![{cairo_array(self.tree_ids)}].span();"
        root_index_cairo = self._generate_root_index_cairo()
        node_index_cairo = self._generate_node_index_cairo()

        return f"{tree_ids_cairo}\n{root_index_cairo}\n{node_index_cairo}"

    def _generate_root_index_cairo(self):
        root_index_lines = [f"    root_index.insert({tid}, {self.root_index[tid]});" for tid in self.tree_ids]
        return "let mut root_index: Felt252Dict<usize> = Default::default();\n" + "\n".join(root_index_lines)

    def _generate_node_index_cairo(self):
        node_index_lines = [f"    node_index.insert({pedersen_hash(int(tid), int(nid))}, {index});"
                            for (tid, nid), index in self.node_index.items()]
        return "let mut node_index: Felt252Dict<usize> = Default::default();\n" + "\n".join(node_index_lines)

def generate_full_cairo_code(params, fixed_point_type="FP16x16"):
    ensemble = TreeEnsemble(fixed_point_type=fixed_point_type, **params)
    tree_specific_code = ensemble.generate_cairo_code()

    # Check for base_values content
    if params['base_values']:
        base_values_cairo = f"let base_values: Option<Span<{fixed_point_type}>> = Option::Some(array![{cairo_array(params['base_values'], fixed_point_type, fixed_point_type)}].span());"
    else:
        base_values_cairo = f"let base_values: Option<Span<{fixed_point_type}>> = Option::None;"

    return f"""
use core::integer::u32;
use xgboost::inputs::input;
use orion::numbers::{{FixedTrait, {fixed_point_type}}};
use orion::operators::tensor::{{Tensor, TensorTrait, {fixed_point_type}Tensor, U32Tensor}};
use orion::operators::ml::tree_ensemble::core::{{NODE_MODES, TreeEnsembleAttributes, TreeEnsemble}};
use orion::operators::ml::tree_ensemble::tree_ensemble_classifier::{{TreeEnsembleClassifier, POST_TRANSFORM, TreeEnsembleClassifierTrait}};
use orion::operators::matrix::{{MutMatrix, MutMatrixImpl}};

fn main() -> u32 {{
    let mut X = input();
    let class_ids: Span<usize> = array![{cairo_array(params['class_ids'])}].span();
    let class_nodeids: Span<usize> = array![{cairo_array(params['class_nodeids'])}].span();
    let class_treeids: Span<usize> = array![{cairo_array(params['class_treeids'])}].span();
    let class_weights: Span<{fixed_point_type}> = array![{cairo_array(params['class_weights'], fixed_point_type, fixed_point_type)}].span();
    let classlabels: Span<usize> = array![{cairo_array(params['classlabels'])}].span();
    let nodes_falsenodeids: Span<usize> = array![{cairo_array(params['nodes_falsenodeids'])}].span();
    let nodes_featureids: Span<usize> = array![{cairo_array(params['nodes_featureids'])}].span();
    let nodes_missing_value_tracks_true: Span<usize> = array![{cairo_array(params['nodes_missing_value_tracks_true'])}].span();
    let nodes_modes: Span<NODE_MODES> = array![{', '.join(['NODE_MODES::' + x for x in params['nodes_modes']])}].span();
    let nodes_nodeids: Span<usize> = array![{cairo_array(params['nodes_nodeids'])}].span();
    let nodes_treeids: Span<usize> = array![{cairo_array(params['nodes_treeids'])}].span();
    let nodes_truenodeids: Span<usize> = array![{cairo_array(params['nodes_truenodeids'])}].span();
    let nodes_values: Span<{fixed_point_type}> = array![{cairo_array(params['nodes_values'], fixed_point_type, fixed_point_type)}].span();
    {base_values_cairo}
    let post_transform = POST_TRANSFORM::{params['post_transform']};

    {tree_specific_code}

    let atts = TreeEnsembleAttributes {{
        nodes_falsenodeids,
        nodes_featureids,
        nodes_missing_value_tracks_true,
        nodes_modes,
        nodes_nodeids,
        nodes_treeids,
        nodes_truenodeids,
        nodes_values
    }};

    let mut ensemble: TreeEnsemble<{fixed_point_type}> = TreeEnsemble {{
        atts, tree_ids, root_index, node_index
    }};

    let mut classifier: TreeEnsembleClassifier<{fixed_point_type}> = TreeEnsembleClassifier {{
        ensemble,
        class_ids,
        class_nodeids,
        class_treeids,
        class_weights,
        classlabels,
        base_values,
        post_transform
    }};

    let (labels, mut scores) = TreeEnsembleClassifierTrait::predict(ref classifier, X);
    let mut arr: Span<usize> = labels;
    let a: usize = *arr.pop_front().unwrap();
    a
}}
    """

# Usage

In [17]:
params = {
    "class_ids": [0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3],
    "class_nodeids": [1, 2, 3, 5, 7, 8, 10, 11, 14, 15, 16, 5, 6, 7, 10, 11, 12, 15, 17, 18, 19, 21, 23, 24, 1, 2, 1, 2, 5, 6, 9, 10, 12, 13, 14, 18, 19, 20, 21, 24, 25, 26, 4, 5, 7, 8, 9, 13, 14, 15, 17, 20, 22, 23, 26, 27, 28, 1, 2, 1, 2, 5, 7, 8, 10, 12, 13, 14, 18, 19, 20, 21, 23, 24, 4, 5, 7, 8, 9, 13, 14, 15, 17, 20, 21, 23, 24, 1, 2, 1, 2, 6, 7, 9, 10, 12, 14, 15, 16, 20, 21, 22, 23, 25, 26, 4, 5, 7, 8, 9, 13, 14, 15, 17, 20, 22, 23, 24, 1, 2, 1, 2, 6, 7, 9, 10, 12, 14, 15, 16, 20, 21, 22, 23, 25, 26, 4, 5, 7, 8, 9, 13, 14, 15, 17, 21, 22, 23, 25, 26, 1, 3, 4, 1, 2, 5, 6, 8, 9, 10, 12, 13, 14, 4, 6, 7, 8, 9, 13, 14, 15, 18, 19, 21, 24, 25, 26, 1, 2, 1, 2, 6, 7, 8, 10, 11, 12, 15, 16, 19, 20, 21, 22, 5, 6, 7, 9, 11, 13, 14, 16, 17, 19, 21, 22, 1, 3, 4, 1, 2, 6, 7, 8, 10, 11, 12, 15, 16, 19, 20, 21, 22, 3, 6, 7, 9, 10, 13, 14, 15, 16, 1, 2, 1, 2, 4, 7, 8, 10, 11, 12, 15, 16, 19, 20, 21, 22, 4, 5, 7, 8, 9, 12, 13, 15, 17, 20, 21, 22, 1, 2, 1, 2, 4, 5, 6, 9, 10, 14, 15, 16, 17, 18, 2, 5, 6, 7, 10, 12, 13, 16, 17, 19, 22, 23, 24, 1, 2],
    "class_treeids": [0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 11, 11, 12, 12, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 15, 15, 16, 16, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 19, 19, 19, 20, 20, 21, 21, 21, 21, 21, 21, 21, 21, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 23, 23, 24, 24, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 27, 27, 27, 28, 28, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 30, 30, 30, 30, 30, 30, 30, 30, 30, 31, 31, 32, 32, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 35, 35, 36, 36, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 39, 39],
    "class_weights": [0.17647060751914978, -0.19546744227409363, 0.2745762765407562, 0.1826086938381195, 0.0, -0.16363637149333954, 0.5368421077728271, -0.02608695812523365, 0.03529411926865578, -0.12000000476837158, -0.17647060751914978, 0.03529411926865578, -0.15000000596046448, 0.12000000476837158, -0.10588236153125763, 0.24000000953674316, 0.4243902564048767, -0.18545454740524292, -0.10588236153125763, 0.03529411926865578, 0.17647060751914978, -0.13846154510974884, -0.02608695812523365, 0.5287128686904907, -0.195389062166214, 0.1826086938381195, 0.15902000665664673, -0.18236583471298218, 0.006204761564731598, 0.31986430287361145, 0.029129553586244583, 0.27554503083229065, 0.008816461078822613, -0.1392686516046524, -0.1500544548034668, 0.24282290041446686, -0.017042117193341255, 0.3519213795661926, -0.023038538172841072, 0.05427384376525879, -0.1112603172659874, -0.16434472799301147, 0.0344294048845768, -0.14162001013755798, -0.02403104119002819, 0.31577974557876587, -0.1694946587085724, -0.10721568763256073, 0.16408854722976685, -0.13714638352394104, -0.11350390315055847, -0.12291733175516129, 0.2853492498397827, 0.03827212378382683, 0.20593757927417755, 0.025907577946782112, 0.3331577777862549, -0.18214519321918488, 0.14501014351844788, 0.1367056518793106, -0.1729452908039093, 0.2510145902633667, 0.23495064675807953, -0.1272488534450531, 0.11457570642232895, 0.0347382090985775, -0.13136233389377594, -0.1429188847541809, 0.20286214351654053, -0.026446590200066566, 0.2688712477684021, -0.019996464252471924, -0.032766032963991165, -0.15382128953933716, 0.02412736788392067, -0.13018612563610077, -0.0266902856528759, 0.24671679735183716, -0.15890821814537048, -0.10435923933982849, 0.13635239005088806, -0.12560059130191803, -0.10402485728263855, 0.01976907067000866, 0.24985870718955994, -0.11412845551967621, 0.187682643532753, -0.17268578708171844, 0.11847954243421555, 0.12341645359992981, -0.16558398306369781, 0.07098324596881866, 0.230085551738739, 0.1975608617067337, -0.11589211970567703, 0.09687639027833939, 0.03348429873585701, -0.12138593941926956, -0.13541941344738007, 0.17489483952522278, -0.03167758882045746, 0.22599880397319794, -0.015596880577504635, -0.019655369222164154, -0.14472194015979767, 0.022149590775370598, -0.1195450946688652, -0.025387009605765343, 0.20259825885295868, -0.14993064105510712, -0.09920255839824677, 0.11511131376028061, -0.11510272324085236, -0.09527357667684555, -0.11308830976486206, 0.19369083642959595, 0.03432764858007431, 0.19767706096172333, -0.1652849316596985, 0.10055933147668839, 0.11082667857408524, -0.15972232818603516, 0.057603560388088226, 0.1995164155960083, 0.17082151770591736, -0.105511374771595, 0.0861908495426178, -0.12113944441080093, 0.025211980566382408, -0.12850631773471832, 0.15224725008010864, -0.034241899847984314, 0.20027507841587067, -0.011911856010556221, -0.006225867196917534, -0.1372060775756836, 0.019379224628210068, -0.11043129116296768, -0.023386459797620773, 0.1733250916004181, -0.14195379614830017, 0.02844543755054474, -0.0010439457837492228, -0.11087937653064728, -0.045292146503925323, -0.0006063051987439394, 0.15625445544719696, 0.2132806032896042, -0.09394722431898117, 0.1348683089017868, -0.16095246374607086, -0.08617648482322693, 0.18108706176280975, 0.10112028568983078, -0.15480269491672516, 0.03997576981782913, 0.12938082218170166, 0.08437880873680115, -0.14704029262065887, 0.1607629358768463, 0.04747379198670387, -0.10739056020975113, -0.1291235387325287, -0.12240501493215561, -0.09891591221094131, 0.14619284868240356, 0.16767343878746033, -0.13430461287498474, 0.023719077929854393, 0.0008078212267719209, -0.10308193415403366, -0.11916369199752808, 0.12314551323652267, 0.1842321753501892, 0.05473789945244789, -0.06331430375576019, 0.16877683997154236, -0.15454581379890442, 0.08099839091300964, 0.09144936501979828, -0.15020686388015747, -0.014884334057569504, 0.13010112941265106, -0.10495828837156296, 0.2198355793952942, 0.012047642841935158, -0.1329028308391571, 0.17761074006557465, 0.05400092899799347, 0.026737814769148827, -0.06347045302391052, 0.11533401906490326, -0.12067287415266037, -0.09142628312110901, 0.10238192230463028, -0.11053753644227982, -0.04464135691523552, 0.14662031829357147, -0.09164517372846603, 0.09022694081068039, -0.0006683502579107881, -0.13208436965942383, -0.09866861253976822, -0.01967775449156761, 0.1571894884109497, -0.15143316984176636, 0.10524453222751617, 0.004876915365457535, 0.08141452074050903, -0.14584296941757202, -0.012311519123613834, 0.11515913903713226, -0.09670499712228775, 0.1949128359556198, 0.008151537738740444, -0.1239418014883995, 0.16589833796024323, 0.044222619384527206, 0.022324711084365845, -0.05429275706410408, 0.10669063776731491, -0.11365038901567459, -0.0653901994228363, 0.10117275267839432, -0.16002774238586426, -0.03624112904071808, 0.16420982778072357, 0.023038508370518684, -0.13711832463741302, 0.09019424021244049, 0.13383401930332184, -0.1469014585018158, 0.06645337492227554, 0.07481520622968674, -0.14151017367839813, -0.0238853320479393, 0.1400492638349533, 0.043577972799539566, 0.1325174868106842, -0.07053554058074951, -0.11534662544727325, 0.1563825160264969, 0.03334147855639458, 0.018541786819696426, -0.05141714587807655, 0.09530365467071533, -0.10619332641363144, 0.00027120221056975424, -0.08401806652545929, -0.012738796882331371, 0.12229666858911514, -0.11519282311201096, 0.022887296974658966, -0.09004124253988266, -0.029741723090410233, 0.15326718986034393, 0.01736612245440483, -0.05817794427275658, 0.14012981951236725, -0.1415504515171051, 0.06344682723283768, 0.0593908466398716, -0.1407313495874405, -0.0373123399913311, 0.10695493966341019, 0.1350965052843094, 0.03544490039348602, -0.12258218973875046, 0.07599421590566635, -0.01258818432688713, 0.12743736803531647, -0.08660592138767242, -0.09850981086492538, -0.10166502743959427, 0.0783810168504715, 0.007186818402260542, -0.09209936857223511, -0.11059971898794174, 0.09433558583259583, 0.027905037626624107, 0.06860843300819397, -0.055248405784368515, 0.13663393259048462, 0.04844236746430397, -0.06016895920038223, 0.12208973616361618, -0.13901002705097198, 0.06150634586811066],
    "classlabels": [0, 1, 2, 3],
    "nodes_falsenodeids": [2, 0, 0, 12, 9, 4, 0, 6, 0, 8, 0, 0, 11, 0, 0, 16, 15, 0, 0, 0, 20, 13, 8, 7, 6, 0, 0, 0, 12, 11, 0, 0, 0, 19, 16, 0, 18, 0, 0, 0, 22, 0, 24, 0, 0, 2, 0, 0, 2, 0, 0, 22, 15, 14, 7, 6, 0, 0, 11, 10, 0, 0, 13, 0, 0, 0, 21, 20, 19, 0, 0, 0, 0, 26, 25, 0, 0, 0, 10, 9, 6, 5, 0, 0, 8, 0, 0, 0, 16, 15, 14, 0, 0, 0, 18, 0, 24, 21, 0, 23, 0, 0, 28, 27, 0, 0, 0, 2, 0, 0, 2, 0, 0, 22, 15, 14, 9, 6, 0, 8, 0, 0, 11, 0, 13, 0, 0, 0, 21, 20, 19, 0, 0, 0, 0, 24, 0, 0, 10, 9, 6, 5, 0, 0, 8, 0, 0, 0, 16, 15, 14, 0, 0, 0, 18, 0, 22, 21, 0, 0, 24, 0, 0, 2, 0, 0, 2, 0, 0, 24, 17, 16, 11, 8, 7, 0, 0, 10, 0, 0, 13, 0, 15, 0, 0, 0, 23, 22, 21, 0, 0, 0, 0, 26, 0, 0, 10, 9, 6, 5, 0, 0, 8, 0, 0, 0, 16, 15, 14, 0, 0, 0, 18, 0, 24, 21, 0, 23, 0, 0, 0, 2, 0, 0, 2, 0, 0, 24, 17, 16, 11, 8, 7, 0, 0, 10, 0, 0, 13, 0, 15, 0, 0, 0, 23, 22, 21, 0, 0, 0, 0, 26, 0, 0, 10, 9, 6, 5, 0, 0, 8, 0, 0, 0, 16, 15, 14, 0, 0, 0, 18, 0, 24, 23, 22, 0, 0, 0, 26, 0, 0, 2, 0, 4, 0, 0, 2, 0, 0, 14, 11, 10, 7, 6, 0, 0, 9, 0, 0, 0, 13, 0, 0, 0, 10, 9, 8, 5, 0, 7, 0, 0, 0, 0, 16, 15, 14, 0, 0, 0, 20, 19, 0, 0, 22, 0, 26, 25, 0, 0, 0, 2, 0, 0, 2, 0, 0, 22, 13, 12, 9, 8, 7, 0, 0, 0, 11, 0, 0, 0, 17, 16, 0, 0, 21, 20, 0, 0, 0, 0, 18, 15, 8, 7, 6, 0, 0, 0, 10, 0, 12, 0, 14, 0, 0, 17, 0, 0, 20, 0, 22, 0, 0, 2, 0, 4, 0, 0, 2, 0, 0, 22, 13, 12, 9, 8, 7, 0, 0, 0, 11, 0, 0, 0, 17, 16, 0, 0, 21, 20, 0, 0, 0, 0, 16, 11, 4, 0, 8, 7, 0, 0, 10, 0, 0, 15, 14, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 22, 13, 12, 5, 0, 9, 8, 0, 0, 11, 0, 0, 0, 17, 16, 0, 0, 21, 20, 0, 0, 0, 0, 10, 9, 6, 5, 0, 0, 8, 0, 0, 0, 14, 13, 0, 0, 16, 0, 18, 0, 22, 21, 0, 0, 0, 2, 0, 0, 2, 0, 0, 18, 7, 6, 5, 0, 0, 0, 11, 10, 0, 0, 17, 16, 15, 0, 0, 0, 0, 0, 8, 3, 0, 7, 6, 0, 0, 0, 14, 11, 0, 13, 0, 0, 18, 17, 0, 0, 20, 0, 24, 23, 0, 0, 0, 2, 0, 0],
    "nodes_featureids": [11, 0, 0, 17, 1, 7, 0, 14, 0, 16, 0, 0, 12, 0, 0, 15, 0, 0, 0, 0, 17, 1, 14, 17, 15, 0, 0, 0, 7, 15, 0, 0, 0, 12, 11, 0, 7, 0, 0, 0, 7, 0, 14, 0, 0, 8, 0, 0, 11, 0, 0, 17, 1, 5, 14, 15, 0, 0, 15, 0, 0, 0, 7, 0, 0, 0, 12, 12, 13, 0, 0, 0, 0, 17, 0, 0, 0, 0, 17, 1, 14, 15, 0, 0, 15, 0, 0, 0, 7, 1, 9, 0, 0, 0, 13, 0, 14, 13, 0, 4, 0, 0, 17, 0, 0, 0, 0, 8, 0, 0, 11, 0, 0, 17, 1, 5, 17, 14, 0, 15, 0, 0, 7, 0, 13, 0, 0, 0, 12, 12, 14, 0, 0, 0, 0, 15, 0, 0, 17, 1, 14, 15, 0, 0, 15, 0, 0, 0, 7, 1, 9, 0, 0, 0, 13, 0, 2, 16, 0, 0, 6, 0, 0, 8, 0, 0, 11, 0, 0, 17, 1, 5, 17, 14, 13, 0, 0, 15, 0, 0, 7, 0, 13, 0, 0, 0, 12, 12, 11, 0, 0, 0, 0, 17, 0, 0, 17, 1, 14, 15, 0, 0, 15, 0, 0, 0, 7, 1, 9, 0, 0, 0, 13, 0, 14, 13, 0, 4, 0, 0, 0, 8, 0, 0, 11, 0, 0, 17, 1, 5, 17, 14, 13, 0, 0, 15, 0, 0, 13, 0, 13, 0, 0, 0, 12, 12, 17, 0, 0, 0, 0, 15, 0, 0, 17, 1, 14, 15, 0, 0, 15, 0, 0, 0, 7, 11, 5, 0, 0, 0, 13, 0, 2, 13, 17, 0, 0, 0, 6, 0, 0, 6, 0, 8, 0, 0, 11, 0, 0, 17, 12, 12, 17, 13, 0, 0, 16, 0, 0, 0, 13, 0, 0, 0, 17, 1, 5, 14, 0, 15, 0, 0, 0, 0, 7, 11, 5, 0, 0, 0, 14, 13, 0, 0, 12, 0, 12, 1, 0, 0, 0, 8, 0, 0, 11, 0, 0, 17, 1, 5, 10, 14, 15, 0, 0, 0, 15, 0, 0, 0, 13, 17, 0, 0, 2, 5, 0, 0, 0, 0, 9, 10, 7, 12, 9, 0, 0, 0, 13, 0, 1, 0, 17, 0, 0, 11, 0, 0, 14, 0, 15, 0, 0, 6, 0, 15, 0, 0, 11, 0, 0, 17, 1, 5, 10, 14, 15, 0, 0, 0, 15, 0, 0, 0, 13, 17, 0, 0, 2, 9, 0, 0, 0, 0, 12, 1, 9, 0, 15, 10, 0, 0, 14, 0, 0, 17, 12, 0, 0, 0, 0, 6, 0, 0, 11, 0, 0, 17, 1, 5, 12, 0, 14, 0, 0, 0, 15, 0, 0, 0, 13, 17, 0, 0, 2, 5, 0, 0, 0, 0, 17, 1, 14, 15, 0, 0, 15, 0, 0, 0, 7, 11, 0, 0, 16, 0, 12, 0, 12, 10, 0, 0, 0, 8, 0, 0, 13, 0, 0, 17, 17, 13, 16, 0, 0, 0, 12, 13, 0, 0, 12, 10, 14, 0, 0, 0, 0, 0, 7, 9, 0, 1, 12, 0, 0, 0, 14, 13, 0, 1, 0, 0, 17, 13, 0, 0, 12, 0, 12, 13, 0, 0, 0, 6, 0, 0],
    "nodes_hitrates": [],
    "nodes_missing_value_tracks_true": [1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0],
    "nodes_modes": ['BRANCH_LT', 'LEAF', 'LEAF', 'BRANCH_LT', 'BRANCH_LT', 'BRANCH_LT', 'LEAF', 'BRANCH_LT', 'LEAF', 'BRANCH_LT', 'LEAF', 'LEAF', 'BRANCH_LT', 'LEAF', 'LEAF', 'BRANCH_LT', 'BRANCH_LT', 'LEAF', 'LEAF', 'LEAF', 'BRANCH_LT', 'BRANCH_LT', 'BRANCH_LT', 'BRANCH_LT', 'BRANCH_LT', 'LEAF', 'LEAF', 'LEAF', 'BRANCH_LT', 'BRANCH_LT', 'LEAF', 'LEAF', 'LEAF', 'BRANCH_LT', 'BRANCH_LT', 'LEAF', 'BRANCH_LT', 'LEAF', 'LEAF', 'LEAF', 'BRANCH_LT', 'LEAF', 'BRANCH_LT', 'LEAF', 'LEAF', 'BRANCH_LT', 'LEAF', 'LEAF', 'BRANCH_LT', 'LEAF', 'LEAF', 'BRANCH_LT', 'BRANCH_LT', 'BRANCH_LT', 'BRANCH_LT', 'BRANCH_LT', 'LEAF', 'LEAF', 'BRANCH_LT', 'BRANCH_LT', 'LEAF', 'LEAF', 'BRANCH_LT', 'LEAF', 'LEAF', 'LEAF', 'BRANCH_LT', 'BRANCH_LT', 'BRANCH_LT', 'LEAF', 'LEAF', 'LEAF', 'LEAF', 'BRANCH_LT', 'BRANCH_LT', 'LEAF', 'LEAF', 'LEAF', 'BRANCH_LT', 'BRANCH_LT', 'BRANCH_LT', 'BRANCH_LT', 'LEAF', 'LEAF', 'BRANCH_LT', 'LEAF', 'LEAF', 'LEAF', 'BRANCH_LT', 'BRANCH_LT', 'BRANCH_LT', 'LEAF', 'LEAF', 'LEAF', 'BRANCH_LT', 'LEAF', 'BRANCH_LT', 'BRANCH_LT', 'LEAF', 'BRANCH_LT', 'LEAF', 'LEAF', 'BRANCH_LT', 'BRANCH_LT', 'LEAF', 'LEAF', 'LEAF', 'BRANCH_LT', 'LEAF', 'LEAF', 'BRANCH_LT', 'LEAF', 'LEAF', 'BRANCH_LT', 'BRANCH_LT', 'BRANCH_LT', 'BRANCH_LT', 'BRANCH_LT', 'LEAF', 'BRANCH_LT', 'LEAF', 'LEAF', 'BRANCH_LT', 'LEAF', 'BRANCH_LT', 'LEAF', 'LEAF', 'LEAF', 'BRANCH_LT', 'BRANCH_LT', 'BRANCH_LT', 'LEAF', 'LEAF', 'LEAF', 'LEAF', 'BRANCH_LT', 'LEAF', 'LEAF', 'BRANCH_LT', 'BRANCH_LT', 'BRANCH_LT', 'BRANCH_LT', 'LEAF', 'LEAF', 'BRANCH_LT', 'LEAF', 'LEAF', 'LEAF', 'BRANCH_LT', 'BRANCH_LT', 'BRANCH_LT', 'LEAF', 'LEAF', 'LEAF', 'BRANCH_LT', 'LEAF', 'BRANCH_LT', 'BRANCH_LT', 'LEAF', 'LEAF', 'BRANCH_LT', 'LEAF', 'LEAF', 'BRANCH_LT', 'LEAF', 'LEAF', 'BRANCH_LT', 'LEAF', 'LEAF', 'BRANCH_LT', 'BRANCH_LT', 'BRANCH_LT', 'BRANCH_LT', 'BRANCH_LT', 'BRANCH_LT', 'LEAF', 'LEAF', 'BRANCH_LT', 'LEAF', 'LEAF', 'BRANCH_LT', 'LEAF', 'BRANCH_LT', 'LEAF', 'LEAF', 'LEAF', 'BRANCH_LT', 'BRANCH_LT', 'BRANCH_LT', 'LEAF', 'LEAF', 'LEAF', 'LEAF', 'BRANCH_LT', 'LEAF', 'LEAF', 'BRANCH_LT', 'BRANCH_LT', 'BRANCH_LT', 'BRANCH_LT', 'LEAF', 'LEAF', 'BRANCH_LT', 'LEAF', 'LEAF', 'LEAF', 'BRANCH_LT', 'BRANCH_LT', 'BRANCH_LT', 'LEAF', 'LEAF', 'LEAF', 'BRANCH_LT', 'LEAF', 'BRANCH_LT', 'BRANCH_LT', 'LEAF', 'BRANCH_LT', 'LEAF', 'LEAF', 'LEAF', 'BRANCH_LT', 'LEAF', 'LEAF', 'BRANCH_LT', 'LEAF', 'LEAF', 'BRANCH_LT', 'BRANCH_LT', 'BRANCH_LT', 'BRANCH_LT', 'BRANCH_LT', 'BRANCH_LT', 'LEAF', 'LEAF', 'BRANCH_LT', 'LEAF', 'LEAF', 'BRANCH_LT', 'LEAF', 'BRANCH_LT', 'LEAF', 'LEAF', 'LEAF', 'BRANCH_LT', 'BRANCH_LT', 'BRANCH_LT', 'LEAF', 'LEAF', 'LEAF', 'LEAF', 'BRANCH_LT', 'LEAF', 'LEAF', 'BRANCH_LT', 'BRANCH_LT', 'BRANCH_LT', 'BRANCH_LT', 'LEAF', 'LEAF', 'BRANCH_LT', 'LEAF', 'LEAF', 'LEAF', 'BRANCH_LT', 'BRANCH_LT', 'BRANCH_LT', 'LEAF', 'LEAF', 'LEAF', 'BRANCH_LT', 'LEAF', 'BRANCH_LT', 'BRANCH_LT', 'BRANCH_LT', 'LEAF', 'LEAF', 'LEAF', 'BRANCH_LT', 'LEAF', 'LEAF', 'BRANCH_LT', 'LEAF', 'BRANCH_LT', 'LEAF', 'LEAF', 'BRANCH_LT', 'LEAF', 'LEAF', 'BRANCH_LT', 'BRANCH_LT', 'BRANCH_LT', 'BRANCH_LT', 'BRANCH_LT', 'LEAF', 'LEAF', 'BRANCH_LT', 'LEAF', 'LEAF', 'LEAF', 'BRANCH_LT', 'LEAF', 'LEAF', 'LEAF', 'BRANCH_LT', 'BRANCH_LT', 'BRANCH_LT', 'BRANCH_LT', 'LEAF', 'BRANCH_LT', 'LEAF', 'LEAF', 'LEAF', 'LEAF', 'BRANCH_LT', 'BRANCH_LT', 'BRANCH_LT', 'LEAF', 'LEAF', 'LEAF', 'BRANCH_LT', 'BRANCH_LT', 'LEAF', 'LEAF', 'BRANCH_LT', 'LEAF', 'BRANCH_LT', 'BRANCH_LT', 'LEAF', 'LEAF', 'LEAF', 'BRANCH_LT', 'LEAF', 'LEAF', 'BRANCH_LT', 'LEAF', 'LEAF', 'BRANCH_LT', 'BRANCH_LT', 'BRANCH_LT', 'BRANCH_LT', 'BRANCH_LT', 'BRANCH_LT', 'LEAF', 'LEAF', 'LEAF', 'BRANCH_LT', 'LEAF', 'LEAF', 'LEAF', 'BRANCH_LT', 'BRANCH_LT', 'LEAF', 'LEAF', 'BRANCH_LT', 'BRANCH_LT', 'LEAF', 'LEAF', 'LEAF', 'LEAF', 'BRANCH_LT', 'BRANCH_LT', 'BRANCH_LT', 'BRANCH_LT', 'BRANCH_LT', 'LEAF', 'LEAF', 'LEAF', 'BRANCH_LT', 'LEAF', 'BRANCH_LT', 'LEAF', 'BRANCH_LT', 'LEAF', 'LEAF', 'BRANCH_LT', 'LEAF', 'LEAF', 'BRANCH_LT', 'LEAF', 'BRANCH_LT', 'LEAF', 'LEAF', 'BRANCH_LT', 'LEAF', 'BRANCH_LT', 'LEAF', 'LEAF', 'BRANCH_LT', 'LEAF', 'LEAF', 'BRANCH_LT', 'BRANCH_LT', 'BRANCH_LT', 'BRANCH_LT', 'BRANCH_LT', 'BRANCH_LT', 'LEAF', 'LEAF', 'LEAF', 'BRANCH_LT', 'LEAF', 'LEAF', 'LEAF', 'BRANCH_LT', 'BRANCH_LT', 'LEAF', 'LEAF', 'BRANCH_LT', 'BRANCH_LT', 'LEAF', 'LEAF', 'LEAF', 'LEAF', 'BRANCH_LT', 'BRANCH_LT', 'BRANCH_LT', 'LEAF', 'BRANCH_LT', 'BRANCH_LT', 'LEAF', 'LEAF', 'BRANCH_LT', 'LEAF', 'LEAF', 'BRANCH_LT', 'BRANCH_LT', 'LEAF', 'LEAF', 'LEAF', 'LEAF', 'BRANCH_LT', 'LEAF', 'LEAF', 'BRANCH_LT', 'LEAF', 'LEAF', 'BRANCH_LT', 'BRANCH_LT', 'BRANCH_LT', 'BRANCH_LT', 'LEAF', 'BRANCH_LT', 'BRANCH_LT', 'LEAF', 'LEAF', 'BRANCH_LT', 'LEAF', 'LEAF', 'LEAF', 'BRANCH_LT', 'BRANCH_LT', 'LEAF', 'LEAF', 'BRANCH_LT', 'BRANCH_LT', 'LEAF', 'LEAF', 'LEAF', 'LEAF', 'BRANCH_LT', 'BRANCH_LT', 'BRANCH_LT', 'BRANCH_LT', 'LEAF', 'LEAF', 'BRANCH_LT', 'LEAF', 'LEAF', 'LEAF', 'BRANCH_LT', 'BRANCH_LT', 'LEAF', 'LEAF', 'BRANCH_LT', 'LEAF', 'BRANCH_LT', 'LEAF', 'BRANCH_LT', 'BRANCH_LT', 'LEAF', 'LEAF', 'LEAF', 'BRANCH_LT', 'LEAF', 'LEAF', 'BRANCH_LT', 'LEAF', 'LEAF', 'BRANCH_LT', 'BRANCH_LT', 'BRANCH_LT', 'BRANCH_LT', 'LEAF', 'LEAF', 'LEAF', 'BRANCH_LT', 'BRANCH_LT', 'LEAF', 'LEAF', 'BRANCH_LT', 'BRANCH_LT', 'BRANCH_LT', 'LEAF', 'LEAF', 'LEAF', 'LEAF', 'LEAF', 'BRANCH_LT', 'BRANCH_LT', 'LEAF', 'BRANCH_LT', 'BRANCH_LT', 'LEAF', 'LEAF', 'LEAF', 'BRANCH_LT', 'BRANCH_LT', 'LEAF', 'BRANCH_LT', 'LEAF', 'LEAF', 'BRANCH_LT', 'BRANCH_LT', 'LEAF', 'LEAF', 'BRANCH_LT', 'LEAF', 'BRANCH_LT', 'BRANCH_LT', 'LEAF', 'LEAF', 'LEAF', 'BRANCH_LT', 'LEAF', 'LEAF'],
    "nodes_nodeids": [0, 1, 2, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 0, 1, 2, 0, 1, 2, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 0, 1, 2, 0, 1, 2, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 0, 1, 2, 0, 1, 2, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 0, 1, 2, 0, 1, 2, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 0, 1, 2, 3, 4, 0, 1, 2, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 0, 1, 2, 0, 1, 2, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 0, 1, 2, 3, 4, 0, 1, 2, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 0, 1, 2, 0, 1, 2, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 0, 1, 2, 0, 1, 2, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 0, 1, 2],
    "nodes_treeids": [0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 12, 12, 12, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 15, 15, 15, 16, 16, 16, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 19, 19, 19, 19, 19, 20, 20, 20, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 23, 23, 23, 24, 24, 24, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 27, 27, 27, 27, 27, 28, 28, 28, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 31, 31, 31, 32, 32, 32, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 35, 35, 35, 36, 36, 36, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 39, 39, 39],
    "nodes_truenodeids": [1, 0, 0, 1, 2, 3, 0, 5, 0, 7, 0, 0, 10, 0, 0, 13, 14, 0, 0, 0, 1, 2, 3, 4, 5, 0, 0, 0, 9, 10, 0, 0, 0, 14, 15, 0, 17, 0, 0, 0, 21, 0, 23, 0, 0, 1, 0, 0, 1, 0, 0, 1, 2, 3, 4, 5, 0, 0, 8, 9, 0, 0, 12, 0, 0, 0, 16, 17, 18, 0, 0, 0, 0, 23, 24, 0, 0, 0, 1, 2, 3, 4, 0, 0, 7, 0, 0, 0, 11, 12, 13, 0, 0, 0, 17, 0, 19, 20, 0, 22, 0, 0, 25, 26, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 2, 3, 4, 5, 0, 7, 0, 0, 10, 0, 12, 0, 0, 0, 16, 17, 18, 0, 0, 0, 0, 23, 0, 0, 1, 2, 3, 4, 0, 0, 7, 0, 0, 0, 11, 12, 13, 0, 0, 0, 17, 0, 19, 20, 0, 0, 23, 0, 0, 1, 0, 0, 1, 0, 0, 1, 2, 3, 4, 5, 6, 0, 0, 9, 0, 0, 12, 0, 14, 0, 0, 0, 18, 19, 20, 0, 0, 0, 0, 25, 0, 0, 1, 2, 3, 4, 0, 0, 7, 0, 0, 0, 11, 12, 13, 0, 0, 0, 17, 0, 19, 20, 0, 22, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 2, 3, 4, 5, 6, 0, 0, 9, 0, 0, 12, 0, 14, 0, 0, 0, 18, 19, 20, 0, 0, 0, 0, 25, 0, 0, 1, 2, 3, 4, 0, 0, 7, 0, 0, 0, 11, 12, 13, 0, 0, 0, 17, 0, 19, 20, 21, 0, 0, 0, 25, 0, 0, 1, 0, 3, 0, 0, 1, 0, 0, 1, 2, 3, 4, 5, 0, 0, 8, 0, 0, 0, 12, 0, 0, 0, 1, 2, 3, 4, 0, 6, 0, 0, 0, 0, 11, 12, 13, 0, 0, 0, 17, 18, 0, 0, 21, 0, 23, 24, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 2, 3, 4, 5, 6, 0, 0, 0, 10, 0, 0, 0, 14, 15, 0, 0, 18, 19, 0, 0, 0, 0, 1, 2, 3, 4, 5, 0, 0, 0, 9, 0, 11, 0, 13, 0, 0, 16, 0, 0, 19, 0, 21, 0, 0, 1, 0, 3, 0, 0, 1, 0, 0, 1, 2, 3, 4, 5, 6, 0, 0, 0, 10, 0, 0, 0, 14, 15, 0, 0, 18, 19, 0, 0, 0, 0, 1, 2, 3, 0, 5, 6, 0, 0, 9, 0, 0, 12, 13, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 2, 3, 4, 0, 6, 7, 0, 0, 10, 0, 0, 0, 14, 15, 0, 0, 18, 19, 0, 0, 0, 0, 1, 2, 3, 4, 0, 0, 7, 0, 0, 0, 11, 12, 0, 0, 15, 0, 17, 0, 19, 20, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 2, 3, 4, 0, 0, 0, 8, 9, 0, 0, 12, 13, 14, 0, 0, 0, 0, 0, 1, 2, 0, 4, 5, 0, 0, 0, 9, 10, 0, 12, 0, 0, 15, 16, 0, 0, 19, 0, 21, 22, 0, 0, 0, 1, 0, 0],
    "nodes_values": [0.1666666716337204, 0.0, 0.0, 0.3571428656578064, 0.5, 0.5, 0.0, 0.25, 0.0, 0.5, 0.0, 0.0, 0.8333333730697632, 0.0, 0.0, 0.5, 0.5, 0.0, 0.0, 0.0, 0.2142857313156128, 0.5, 0.25, 0.0714285746216774, 0.5, 0.0, 0.0, 0.0, 0.5, 0.5, 0.0, 0.0, 0.0, 0.8333333730697632, 0.8333333730697632, 0.0, 0.5, 0.0, 0.0, 0.0, 0.5, 0.0, 0.75, 0.0, 0.0, 0.25, 0.0, 0.0, 0.1666666716337204, 0.0, 0.0, 0.3571428656578064, 0.5, 0.5, 0.25, 0.5, 0.0, 0.0, 0.5, 0.5, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.8333333730697632, 0.5, 0.5, 0.0, 0.0, 0.0, 0.0, 0.5, 0.5, 0.0, 0.0, 0.0, 0.0714285746216774, 0.5, 0.25, 0.5, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.5, 0.5, 0.1666666716337204, 0.0, 0.0, 0.0, 0.2142857313156128, 0.0, 0.75, 0.5, 0.0, 0.5, 0.0, 0.0, 0.2142857313156128, 0.6666666865348816, 0.0, 0.0, 0.0, 0.25, 0.0, 0.0, 0.1666666716337204, 0.0, 0.0, 0.3571428656578064, 0.5, 0.5, 0.0714285746216774, 0.25, 0.0, 0.5, 0.0, 0.0, 0.5, 0.0, 0.5, 0.0, 0.0, 0.0, 0.8333333730697632, 0.5, 0.75, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0714285746216774, 0.5, 0.25, 0.5, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.5, 0.5, 0.1666666716337204, 0.0, 0.0, 0.0, 0.2142857313156128, 0.0, 0.5, 0.5, 0.0, 0.0, 0.5, 0.0, 0.0, 0.25, 0.0, 0.0, 0.1666666716337204, 0.0, 0.0, 0.3571428656578064, 0.5, 0.5, 0.0714285746216774, 0.25, 0.2142857313156128, 0.0, 0.0, 0.5, 0.0, 0.0, 0.5, 0.0, 0.5, 0.0, 0.0, 0.0, 0.8333333730697632, 0.5, 0.6666666865348816, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0714285746216774, 0.5, 0.25, 0.5, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.5, 0.5, 0.1666666716337204, 0.0, 0.0, 0.0, 0.2142857313156128, 0.0, 0.75, 0.5, 0.0, 0.5, 0.0, 0.0, 0.0, 0.25, 0.0, 0.0, 0.1666666716337204, 0.0, 0.0, 0.3571428656578064, 0.5, 0.5, 0.0714285746216774, 0.25, 0.2142857313156128, 0.0, 0.0, 0.5, 0.0, 0.0, 0.5, 0.0, 0.9285714626312256, 0.0, 0.0, 0.0, 0.8333333730697632, 0.5, 0.0714285746216774, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0714285746216774, 0.5, 0.25, 0.5, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.5, 0.8333333730697632, 0.5, 0.0, 0.0, 0.0, 0.3571428656578064, 0.0, 0.5, 0.7857142686843872, 0.2142857313156128, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.5, 0.0, 0.25, 0.0, 0.0, 0.1666666716337204, 0.0, 0.0, 0.5, 0.8333333730697632, 0.5, 0.0714285746216774, 0.2142857313156128, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.7857142686843872, 0.0, 0.0, 0.0, 0.0714285746216774, 0.5, 0.5, 0.25, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.5, 0.8333333730697632, 0.5, 0.0, 0.0, 0.0, 0.75, 0.5, 0.0, 0.0, 0.5, 0.0, 0.8333333730697632, 0.5, 0.0, 0.0, 0.0, 0.25, 0.0, 0.0, 0.1666666716337204, 0.0, 0.0, 0.5, 0.5, 0.5, 0.75, 0.75, 0.5, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.7857142686843872, 0.2142857313156128, 0.0, 0.0, 0.5, 0.5, 0.0, 0.0, 0.0, 0.0, 0.5, 0.75, 0.5, 0.5, 0.1666666716337204, 0.0, 0.0, 0.0, 0.2142857313156128, 0.0, 0.5, 0.0, 0.0714285746216774, 0.0, 0.0, 0.5, 0.0, 0.0, 0.75, 0.0, 0.5, 0.0, 0.0, 0.5, 0.0, 0.5, 0.0, 0.0, 0.1666666716337204, 0.0, 0.0, 0.5, 0.5, 0.5, 0.75, 0.75, 0.5, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.7857142686843872, 0.2142857313156128, 0.0, 0.0, 0.5, 0.5, 0.0, 0.0, 0.0, 0.0, 0.8333333730697632, 0.5, 0.1666666716337204, 0.0, 0.5, 0.75, 0.0, 0.0, 0.25, 0.0, 0.0, 0.3571428656578064, 0.5, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.1666666716337204, 0.0, 0.0, 0.5, 0.5, 0.5, 0.1666666716337204, 0.0, 0.25, 0.5, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.7857142686843872, 0.2142857313156128, 0.0, 0.0, 0.5, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0714285746216774, 0.5, 0.25, 0.5, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.5, 0.8333333730697632, 0.0, 0.0, 0.5, 0.0, 0.5, 0.0, 0.8333333730697632, 0.75, 0.0, 0.0, 0.0, 0.25, 0.0, 0.0, 0.2142857313156128, 0.0, 0.0, 0.5, 0.0714285746216774, 0.3571428656578064, 0.5, 0.0, 0.0, 0.0, 0.5, 0.5, 0.0, 0.0, 0.8333333730697632, 0.75, 0.75, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.1666666716337204, 0.0, 0.5, 0.5, 0.0, 0.0, 0.0, 0.75, 0.5, 0.0, 0.5, 0.0, 0.0, 0.0714285746216774, 0.5, 0.0, 0.0, 0.5, 0.0, 0.8333333730697632, 0.6428571939468384, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0],
    "base_values": [0.5, 0.5, 0.5, 0.5],
    "post_transform": "SOFTMAX",
}

fixed_point_type = "FP16x16"  # You can change this to FP8x23, FP32x32, FP64x64
full_cairo_code = generate_full_cairo_code(params, fixed_point_type)
print(full_cairo_code)


use core::integer::u32;
use xgboost::inputs::input;
use orion::numbers::{FixedTrait, FP16x16};
use orion::operators::tensor::{Tensor, TensorTrait, FP16x16Tensor, U32Tensor};
use orion::operators::ml::tree_ensemble::core::{NODE_MODES, TreeEnsembleAttributes, TreeEnsemble};
use orion::operators::ml::tree_ensemble::tree_ensemble_classifier::{TreeEnsembleClassifier, POST_TRANSFORM, TreeEnsembleClassifierTrait};
use orion::operators::matrix::{MutMatrix, MutMatrixImpl};

fn main() -> u32 {
    let mut X = input();
    let class_ids: Span<usize> = array![0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0